In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [5]:
from TSFEA import *

In [6]:
tsfea = TSFEA()

# Get Earnings Number to Homogenize Dates across Tickers

In [45]:
homogenized_earnings = tsfea.homogenize_earnings_dates('AllEarningsFmt.csv')
homogenized_earnings.head()

CUSIP       EDATE        LFRI  PrePRC  OpmPRC  PostPRC  \
TICKER ANNDATS                                                               
000R   20191106  14163310  2019-11-06  2019-11-01   11.96   12.02    12.35   
       20190806  14163310  2019-08-06  2019-08-02   10.22   10.71     7.73   
       20190509  14163310  2019-05-09  2019-05-03   16.32   16.40    14.94   
       20190307  14163310  2019-03-07  2019-03-01   24.17   25.54    22.80   
       20181108  14163310  2018-11-08  2018-11-02   18.00   18.10    18.85   

                 ERNUM  
TICKER ANNDATS          
000R   20191106     15  
       20190806     14  
       20190509     13  
       20190307     12  
       20181108     11

# Reconcile Estimates Dates

In [26]:
reconciled_estimates = tsfea.reconcile_estimates_with_earnings(
    'InitialEstimatesData20182022.csv',
    homogenized_earnings
)

In [21]:
reconciled_estimates.head()

OFTIC     CUSIP          CNAME  STATPERS MEASURE FISCALP  FPI  \
TICKER ERNUM                                                                 
00AI   1      WATT  29272C10  ENERGOUS CORP  20210520     EPS     QTR    6   
       1      WATT  29272C10  ENERGOUS CORP  20210617     EPS     QTR    6   
       1      WATT  29272C10  ENERGOUS CORP  20210715     EPS     QTR    6   
       1      WATT  29272C10  ENERGOUS CORP  20210318     EPS     QTR    7   
       1      WATT  29272C10  ENERGOUS CORP  20210415     EPS     QTR    7   

             ESTFLAG  NUMEST  NUMUP  NUMDOWN  MEDEST  MEANEST  STDEV  HIGHEST  \
TICKER ERNUM                                                                    
00AI   1           P       2      1        1   -0.12    -0.12   0.01    -0.11   
       1           P       2      0        0   -0.12    -0.12   0.01    -0.11   
       1           P       2      0        0   -0.12    -0.12   0.01    -0.11   
       1           P       2      1        0   -0.12    -0.12   0.00    -0.12   
       1           P       2      0        0   -0.12    -0.12   0.00    -0.12   

              LOWEST  
TICKER ERNUM          
00AI   1       -0.13  
       1       -0.13  
       1       -0.13  
       1       -0.12  
       1       -0.12

# Reconcile Returns

In [ ]:
reconciled_returns = tsfea.reconcile_returns_with_earnings(
    'InitialEstimatesData20182022.csv',
    homogenized_earnings
)

In [ ]:
reconciled_returns.head()

# Feature Extraction

In [29]:
medest_features = tsfea.extract_features_from_column(
    reconciled_estimates,
    'MEDEST'
)

In [30]:
medest_features.head()

MEDESTDN_HistogramMode_5  MEDESTDN_HistogramMode_10  \
TICKER ERNUM                                                        
00AI   0                  3.980689e-01               9.056067e-01   
       2                           NaN                        NaN   
       3                 -8.881784e-16              -8.881784e-16   
       4                  8.490331e-01               9.515026e-01   
       5                 -5.250000e-01              -6.375000e-01   

              MEDESTCO_f1ecac  MEDESTCO_FirstMin_ac  \
TICKER ERNUM                                          
00AI   0             2.590830                   7.0   
       2             0.000000                   0.0   
       3             0.505696                   2.0   
       4             0.677410                   2.0   
       5             1.704643                   6.0   

              MEDESTCO_HistogramAMI_even_2_5  MEDESTCO_trev_1_num  \
TICKER ERNUM                                                        
00AI   0                            0.547993             0.188513   
       2                                 NaN                  NaN   
       3                            0.693147             0.000000   
       4                            0.500402            -1.423097   
       5                            0.389048             1.265625   

              MEDESTMD_hrv_classic_pnn40  \
TICKER ERNUM                               
00AI   0                        0.214286   
       2                             NaN   
       3                        0.666667   
       4                        0.666667   
       5                        0.333333   

              MEDESTSB_BinaryStats_mean_longstretch1  \
TICKER ERNUM                                           
00AI   0                                        24.0   
       2                                         NaN   
       3                                         2.0   
       4                                         2.0   
       5                                         3.0   

              MEDESTSB_TransitionMatrix_3ac_sumdiagcov  \
TICKER ERNUM                                             
00AI   0                                      0.027778   
       2                                           NaN   
       3                                      0.074074   
       4                                      0.074074   
       5                                      0.166667   

              MEDESTPD_PeriodicityWang_th0_01  ...  \
TICKER ERNUM                                   ...   
00AI   0                                 14.0  ...   
       2                                  0.0  ...   
       3                                  0.0  ...   
       4                                  0.0  ...   
       5                                  2.0  ...   

              MEDESTFC_LocalSimple_mean1_tauresrat  \
TICKER ERNUM                                         
00AI   0                                  0.142857   
       2                                       NaN   
       3                                  1.000000   
       4                                  1.000000   
       5                                  0.250000   

              MEDESTDN_OutlierInclude_p_001_mdrmd  \
TICKER ERNUM                                        
00AI   0                                 0.635294   
       2                                      NaN   
       3                                 0.250000   
       4                                 0.428571   
       5                                 0.700000   

              MEDESTDN_OutlierInclude_n_001_mdrmd  \
TICKER ERNUM                                        
00AI   0                                -0.270588   
       2                                      NaN   
       3                                 0.250000   
       4                                 0.000000   
       5                                -0.300000   

              MEDESTSP_Summaries_welch_rect_area_

In [ ]:
return_features = tsfea.extract_features_from_column(
    reconciled_returns,
    'MEDEST'
)

In [ ]:
return_features.head()

# Prediction Test

### Add Models

In [62]:
tsfea.models = {
    'XGBClf': XGBClassifier(eval_metric='error')
}

tsfea.hyparam_space = {
    'XGBClf': {
        'max_depth': scope.int(hp.quniform("max_depth", 3, 18, 1)),
        'gamma': hp.uniform ('gamma', 0, 9),
        'reg_alpha' : hp.uniform('reg_alpha', 0, 1),
        'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
        'min_child_weight' : scope.int(hp.quniform('min_child_weight', 0, 10, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 100, 400, 1))
    }
}

for more details on defining hyperparameter spaces, visit http://hyperopt.github.io/hyperopt/getting-started/search_spaces/, or google hyperparameter tuning [ml model] hyperopt.

In [99]:
target_earnings = homogenized_earnings.droplevel(
    1
).set_index(
    "ERNUM",
    append=True
).sort_index()

target_earnings["target"] = pd.to_numeric((target_earnings.PostPRC / target_earnings.PrePRC - 1) >= 0)
target_earnings = pd.DataFrame(
        target_earnings.loc[
        :,
        "target"
    ]
)

In [100]:
df = target_earnings.join(
    medest_features, 
    how='inner'
).groupby(
    level=[0, 1]
).fillna(
    method='ffill'
).fillna(
    0
)
df.head()

target  MEDESTDN_HistogramMode_5  MEDESTDN_HistogramMode_10  \
TICKER ERNUM                                                                
00AI   2       False              0.000000e+00               0.000000e+00   
       3        True             -8.881784e-16              -8.881784e-16   
       4       False              8.490331e-01               9.515026e-01   
       5       False             -5.250000e-01              -6.375000e-01   
       6       False             -5.114741e-01              -6.373491e-01   

              MEDESTCO_f1ecac  MEDESTCO_FirstMin_ac  \
TICKER ERNUM                                          
00AI   2             0.000000                   0.0   
       3             0.505696                   2.0   
       4             0.677410                   2.0   
       5             1.704643                   6.0   
       6             2.362916                   9.0   

              MEDESTCO_HistogramAMI_even_2_5  MEDESTCO_trev_1_num  \
TICKER ERNUM                                                        
00AI   2                            0.000000             0.000000   
       3                            0.693147             0.000000   
       4                            0.500402            -1.423097   
       5                            0.389048             1.265625   
       6                            0.512778             0.445191   

              MEDESTMD_hrv_classic_pnn40  \
TICKER ERNUM                               
00AI   2                        0.000000   
       3                        0.666667   
       4                        0.666667   
       5                        0.333333   
       6                        0.333333   

              MEDESTSB_BinaryStats_mean_longstretch1  \
TICKER ERNUM                                           
00AI   2                                         0.0   
       3                                         2.0   
       4                                         2.0   
       5                                         3.0   
       6                                         5.0   

              MEDESTSB_TransitionMatrix_3ac_sumdiagcov  ...  \
TICKER ERNUM                                            ...   
00AI   2                                      0.000000  ...   
       3                                      0.074074  ...   
       4                                      0.074074  ...   
       5                                      0.166667  ...   
       6                                      0.166667  ...   

              MEDESTFC_LocalSimple_mean1_tauresrat  \
TICKER ERNUM                                         
00AI   2                                      0.00   
       3                                      1.00   
       4                                      1.00   
       5                                      0.25   
       6                                      0.20   

              MEDESTDN_OutlierInclude_p_001_mdrmd  \
TICKER ERNUM                                        
00AI   2                                 0.000000   
       3                                 0.250000   
       4                                 0.428571   
       5                                 0.700000   
       6                                 0.846154   

              MEDESTDN_OutlierInclude_n_001_mdrmd  \
TICKER ERNUM                                        
00AI   2                                 0.000000   
       3                                 0.250000   
       4                                 0.000000   
       5                                -0.300000   
       6                                -0.384615   

              MEDESTSP_Summaries_welch_rect_area_5_1  \
TICKER ERNUM                                           
00AI   2                                0.000000e+00   
       3                                0.000000e+00   
       4                                8.804251e-34   
       5                                2.773339e-33   
 

In [101]:
(
    X_train, 
    X_test, 
    y_train, 
    y_test
) = (
    df.loc[(slice(None), slice(1, 13)), df.columns != "target"],
    df.loc[(slice(None), slice(14, 16)), df.columns != "target"],
    df.loc[(slice(None), slice(1, 13)), "target"],
    df.loc[(slice(None), slice(14, 16)), "target"],
)

In [102]:
y_pred = tsfea.tune_train_predict(
    X_train,
    X_test,
    y_train,
    y_test,
    'XGBClf',
    15,
    50
)

100%|██████████| 50/50 [01:30<00:00,  1.81s/trial, best loss: -0.6397924462264405]


In [104]:
metrics.f1_score(y_pred, y_test)

0.5709312445604874

# Toy Prediction Test

In [94]:
from sklearn.datasets import load_breast_cancer

In [95]:
data = load_breast_cancer()
X, y = data.data, data.target

In [150]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [151]:
y_pred = tsfea.tune_train_predict(
    X_train,
    X_test,
    y_train,
    y_test,
    'XGBClf',
    15,
    50
)

100%|██████████| 50/50 [00:35<00:00,  1.41trial/s, best loss: -0.9479826194909994]
